
# Message Authentication Code (MAC)
A MAC involves a hashing being signed by a shared secret key between Bob and Alice. For this, we define a given hashing method, such as SHA-256, and a given MAC type.

## HMAC
HMAC (hash-based message authentication code) supports the usage of a key to hash data. This key is kept secret between Bob and Alice, and can be used to authentication both the data and that the sender still knows the secret. Overall HMAC can be used with a range of different hashing methods, such as MD5, SHA-1, SHA-256 (SHA-2) and SHA-3. 

<img src='graphics/g_mac_01.png' width="800px">

For HMAC-BLAKE2, we can have:

<img src='graphics/g_mac_02.png' width="800px">


The code to implement this is:


In [2]:
# 05_01.py
# https://asecuritysite.com/hazmat/hashnew2

from cryptography.hazmat.primitives import hashes, hmac
import binascii
from cryptography.hazmat.backends import default_backend

st = "The quick brown fox jumps over the lazy dog"
hex=False
showhex="No"
k="key"

def show_hmac(name,type,data,key):
  digest = hmac.HMAC(key, type,backend=default_backend())
  digest.update(data)
  res=digest.finalize()
  hex=binascii.b2a_hex(res).decode()
  b64=binascii.b2a_base64(res).decode()
  print (f"HMAC-{name}: {hex} {b64}")
  
  digest2 = hmac.HMAC(key, type,backend=default_backend())
  digest2.update(data)
  rtn=digest2.verify(res)
  if (rtn==None): print("HMAC Verified")

try:
	if (hex==True): data = binascii.a2b_hex(st)
	else: data=st.encode()

	if (hex==True): key = binascii.a2b_hex(k)
	else: key=k.encode()

	print ("Data: ",st)
	print (" Hex: ",binascii.b2a_hex(data).decode())
	print ("Key: ",k)
	print (" Hex: ",binascii.b2a_hex(key).decode())
	print()

	show_hmac("SHA256",hashes.SHA256(),data,key)

except Exception as e:
    print(e)

Data:  The quick brown fox jumps over the lazy dog
 Hex:  54686520717569636b2062726f776e20666f78206a756d7073206f76657220746865206c617a7920646f67
Key:  key
 Hex:  6b6579

HMAC-SHA256: f7bc83f430538424b13298e6aa6fb143ef4d59a14946175997479dbc2d1a3cd8 97yD9DBThCSxMpjmqm+xQ+9NWaFJRhdZl0edvC0aPNg=

HMAC Verified


> Verify that the HMAC signature works with the given message.

> Modify the program for HMAC-MD5 and verify that it works.

> Modify the program for HMAC-SHA1 and verify that it works.

> Modify the program for HMAC-SHA512 and verify that it works.

> Observe how the signature varies for different hashing methods.

## Poly1305
For a stream cipher, it is important that we have a MAC, so that we can detect any changes in the bits in the cipher. ChaCha20 typically uses Poly1305 for its MAC. 

<img src='graphics/g_mac_05.png' width="800px">

In [9]:
# 05_02.py
# https://asecuritysite.com/hazmat/hashnew27
import os
import sys
import binascii

from cryptography.hazmat.primitives import poly1305


message = "message"

if (len(sys.argv)>1):
	message=str(sys.argv[1])


message=message.encode()
key = os.urandom(32)

c  = poly1305.Poly1305(key)

c.update(message)


signature = c.finalize()

print (f"Message: {message.decode()}" )

print (f"Key: {binascii.b2a_hex(key).decode()}")

print (f"\nPoly1305 tag (16 bytes): {binascii.b2a_hex(signature).decode()}")

c  = poly1305.Poly1305(key)
c.update(message)

try:
  rtn=c.verify(signature)
  print ("Signature matches")
except:
  print ("Signature does not match")



print ("\n\n--- Generating with alternative method ---")
tag=poly1305.Poly1305.generate_tag(key,message)
print (binascii.b2a_hex(tag).decode())


Message: --f=/Users/billbuchanan/Library/Jupyter/runtime/kernel-v2-71602TT5d9wYK6Zod.json
Key: a84edba3b95371406aa09d7f3c0e36b8fb1ac7808dd204078f0660cf9dfe3834

Poly1305 tag (16 bytes): cf7fd40e03581a380911434fc41d40c2
Signature matches


--- Generating with alternative method ---
cf7fd40e03581a380911434fc41d40c2


> Verify the the operation of the Poly1305 in this program.

> What is the size of the key used in this example?

> How many bits does the Poly1305 tag have?

## CMAC
CMACs (Cipher-based message authentication codes) create a message authentication codes (MACs) using a block cipher and a secret key. They differ from HMACs in that they use a symmetric key method for the MACs rather than a hashing method. In the following, we use the AES method to create the MAC:

<img src='graphics/g_mac_03.png' width="800px">

In [11]:
# 05_03.py
# https://asecuritysite.com/hazmat/hashnew26
import os
import sys
import binascii

from cryptography.hazmat.primitives import cmac

from cryptography.hazmat.primitives.ciphers import algorithms

message = "message"
ctype="TripleDES"

if (len(sys.argv)>1):
	message=str(sys.argv[1])
if (len(sys.argv)>2):
	ctype=str(sys.argv[2])

message=message.encode()
key = os.urandom(32)

c = cmac.CMAC(algorithms.AES(key),backend=default_backend())



c.update(message)

c_copy = c.copy() 

signature = c.finalize()

print (f"Message: {message.decode()}" )
print (f"Type: {ctype}" )
print (f"Key: {binascii.b2a_hex(key).decode()}")

print (f"CMAC signature: {binascii.b2a_hex(signature).decode()}")

try:
  rtn=c_copy.verify(signature)
  print ("Signature matches")
except:
  print ("Signature does not match")


Message: --f=/Users/billbuchanan/Library/Jupyter/runtime/kernel-v2-71602TT5d9wYK6Zod.json
Type: TripleDES
Key: 56837cb6ddfc06db9f55553e653628cc3cdfaf738f45deee83698fdf60bd1c87
CMAC signature: 58118c98fa654fb7a42a392b5df31c26
Signature matches


> Verify the operation of the code.

> Rather than using AES, implement the code using 3DES.

> Rather than using AES, implement the code using Blowfish.

## Generate HMAC with OpenSSL
We can use OpenSSL to generate MACs. For HMAC, we can generate:

```
Message: Hello
Password: test
Hash: sha256
========================
HMAC:
Windows command: "echo set /p="Hello" | openssl dgst -sha256 -hmac test"
Linux command: "echo -n "Hello" | openssl dgst -sha256 -hmac test"
Result:
e2bb88dfaff4fea88d7dc7a00d18b79bb971653de0fe54c7f3985e4daa1e6a25
```

>> With OpenSSL, generate the HMAC signature for a message of "Goodbye" and a key of "qwerty123". Use a hash of SHA1, SHA256, and SHA3-256. Check your answer [here](https://asecuritysite.com/mac/openssl_hmac).